# Resumo: BERT e Suas Variações

## 1. Por que o BERT funciona?

- **Arquitetura Transformer Bidirecional**  
  - Processa cada token considerando contexto à esquerda e à direita simultaneamente.  
  - Captura relações de longo alcance sem limitação de janela fixa.

- **Pré-treinamento em Grande Escala**  
  1. **Masked Language Modeling (MLM):** 15 % dos tokens mascarados; o modelo aprende a prever cada palavra a partir do contexto.  
  2. **Next Sentence Prediction (NSP):** aprende relações entre sentenças, útil para tarefas de pares (Q&A, inferência).

- **Embeddings Contextuais**  
  - O mesmo token (“bank”) tem vetores diferentes em “river bank” vs “bank account”.  
  - Rica representação semântica que reflete sentido, sintaxe e posição.

---

## 2. Vantagens do BERT

1. **Contextualização Dinâmica:** desambiguação de polissemia.  
2. **Transfer Learning:** “fine-tune” rápido para tarefas específicas (classificação, NER, Q&A).  
3. **Melhor Desempenho em Tarefas Complexas:** especialmente onde contexto profundo importa (análise de sentimentos nuance, perguntas e respostas).

---

## 3. Limitações do BERT

1. **Custo Computacional Elevado:**  
   - Extração de embeddings requer GPU e muita memória (768–1024 dimensões).  
2. **Latência de Inferência:**  
   - Tokenização, padding e passagem pela rede são lentos, difícil deploy em tempo real de alta escala.  
3. **Tamanho e Versão do Modelo:**  
   - Modelos grandes (BERT-Base, BERT-Large) não cabem em dispositivos edge; versões menores (DistilBERT, TinyBERT) sacrificam performance.
4. **Dados Limitados no Fine-tuning:**  
   - Com poucos exemplos, overfitting se torna risco; técnicas como “layer freezing” ajudam.

---

## 4. Elementos e Hiperparâmetros Principais

| Elemento / Parâmetro           | Descrição & Variações                                                                                              | Quando usar / Exemplo                                                   |
|--------------------------------|---------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------|
| **Modelo-base**                | – `bert-base-uncased` (12 camadas, 768 hidden, 12 heads) <br>– `bert-large-uncased` (24 camadas, 1024 hidden, 16 heads) | Base para maioria das tarefas; Large para Q&A exigente em contexto.      |
| **Tokenizer**                  | – WordPiece, Byte-Pair Encoding, SentencePiece                                                                     | WordPiece é padrão BERT; SentencePiece (em mBERT/ALBERT) capta melhor idiomas com vocabulário diverso. |
| **Max Sequence Length**        | Número máximo de tokens (p.ex. 128, 256, 512)                                                                        | 128–256 para tweets/SMS; 512 para documentos longos; cuidado com memória. |
| **Batch Size**                 | Tamanho de lote (p.ex. 8, 16, 32)                                                                                   | GPU limitada → batch pequeno; usar gradient accumulation se precisar.    |
| **Learning Rate**              | Tipicamente `2e-5` a `5e-5` para fine-tuning                                                                         | Taxa baixa para evitar “destruir” pré-treinamento; experimente 3e-5.      |
| **Number of Epochs**           | 2–4 épocas geralmente suficientes                                                                                    | 3 épocas costuma equilibrar aprendizado e overfitting.                   |
| **Warmup Steps**               | Proporção inicial (10 % dos passos) para aquecer learning rate                                                      | Ajuda a estabilizar otimização; configure `warmup_steps = total_steps * 0.1`. |
| **Dropout**                    | Regularização (p.ex. 0.1)                                                                                            | Mantém coesão de representações; use valor padrão 0.1 ou ajuste leve.    |
| **Layer Freezing**             | Congelar primeiras N camadas para reduzir overfitting                                                               | Com poucos dados de fine-tune, congele 6 camadas iniciais e treine só as top. |

---

## 5. Exemplos Comparativos

| Tarefa                      | Configuração Recomendada                                | Modelo Alternativo         |
|-----------------------------|----------------------------------------------------------|----------------------------|
| Classificação de Sentimentos (tweets) | BERT-Base, max_len=128, batch=16, lr=3e-5, 3 épocas         | DistilBERT (sem GPU)       |
| Q&A em Documentos Jurídicos | BERT-Large, max_len=512, batch=8, lr=2e-5, 4 épocas       | RoBERTa-Large              |
| NER em Dados Clínicos       | BERT-Base-cased, tokenizer cased, max_len=256, lr=3e-5    | BioBERT / ClinicalBERT     |

---

> **Dica final:** ajuste hiperparâmetros com grid search leve usando validação cruzada e considere versões menores (DistilBERT, ALBERT) para deploy em produção com recurso limitado.  

## Exemplos de Treinamento e Classificação com BERT

A seguir, veremos dois exemplos completos de fine-tuning de BERT em um dataset público de classificação de texto (IMDb) — um rodando em **CPU** e outro aproveitando **GPU (CUDA)**, se disponível.

---

# Passo 1: Instalar e importar dependências

In [6]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 1. Preparação do Dataset
# -------------------------
ds = load_dataset("imdb")
# Amostra pequena para exemplo
train_ds = ds["train"].shuffle(seed=42).select(range(2000))
test_ds  = ds["test"].shuffle(seed=42).select(range(500))

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess(batch):
    enc = tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt",
    )
    return {
        "input_ids":  enc["input_ids"],
        "attention_mask": enc["attention_mask"],
        "labels": torch.tensor(batch["label"]),
    }

train_ds = train_ds.map(preprocess, batched=True, remove_columns=["text"])
test_ds  = test_ds.map(preprocess,  batched=True, remove_columns=["text"])

train_ds.set_format("torch")
test_ds.set_format("torch")

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=16)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

# 2. Configuração do Modelo e Otimizador

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 3. Função de Avaliação

In [ ]:
def evaluate(loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())
    preds = torch.cat(all_preds)
    labs  = torch.cat(all_labels)
    acc = accuracy_score(labs, preds)
    p, r, f1, _ = precision_recall_fscore_support(labs, preds, average="binary")
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}


# 4. Loop de Treino (26min na cpu, 3min no cuda)


In [9]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    metrics = evaluate(test_loader)
    print(f"Época {epoch+1}/{num_epochs} →", metrics)

Época 1/3 → {'accuracy': 0.864, 'precision': 0.8869565217391304, 'recall': 0.8292682926829268, 'f1': 0.8571428571428571}
Época 2/3 → {'accuracy': 0.86, 'precision': 0.8410852713178295, 'recall': 0.8821138211382114, 'f1': 0.8611111111111112}
Época 3/3 → {'accuracy': 0.798, 'precision': 0.7203647416413373, 'recall': 0.9634146341463414, 'f1': 0.8243478260869566}


# 5. Inferência em Novos Textos


In [10]:
samples = [
    "An amazing movie, truly enjoyed every minute!",
    "Terrible plot, I would not recommend it to anyone."
]
enc = tokenizer(samples, padding=True, truncation=True, max_length=128, return_tensors="pt")
enc = {k: v.to(device) for k, v in enc.items()}

model.eval()
with torch.no_grad():
    outputs = model(**enc)
    probs = torch.softmax(outputs.logits, dim=-1)
    preds = torch.argmax(probs, dim=-1)

for text, pred, prob in zip(samples, preds.cpu(), probs.cpu()):
    label = "positive" if pred==1 else "negative"
    print(f"Texto: {text}\n→ {label} (confiança {prob[pred]:.2f})\n")

Texto: An amazing movie, truly enjoyed every minute!
→ positive (confiança 1.00)

Texto: Terrible plot, I would not recommend it to anyone.
→ negative (confiança 0.98)

